# User reviews 

The following notebook contains an ETL process to which a file containing data from the video games platform STEAM was subjected. The file contains reviews of different products of the platform as well as related information such as, the date the review was made, if other users found it helpful or not, etc.

In [1]:
#Needed_Libraries

import pandas as pd
import numpy as np
import json
import ast
from etl_functions import general_information, load_json_lines, duplicated_values, expand_json_column

We start by opening the file and getting some general insights about the data.

In [58]:
#Opening the file
df_reviews=load_json_lines("data/australian_user_reviews.json")

In [59]:
df_reviews.head(5)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [60]:
df_reviews.shape

(25799, 3)

In [61]:
general_information(df_reviews)

,column,data_type,missing_values,missing_values_percentage
0,user_id,<class 'str'>,0,0.0
1,user_url,<class 'str'>,0,0.0
2,reviews,<class 'str'>,0,0.0


In [62]:
#Looking for duplicated values
duplicated_values(df_reviews,"user_id")

,user_id,user_url,reviews
12888,05041129,http://steamcommunity.com/id/05041129,"[{'funny': '', 'posted': 'Posted May 18, 2015...."
5250,05041129,http://steamcommunity.com/id/05041129,"[{'funny': '', 'posted': 'Posted May 18, 2015...."
3133,111222333444555666888,http://steamcommunity.com/id/11122233344455566...,"[{'funny': '', 'posted': 'Posted December 22, ..."
3134,111222333444555666888,http://steamcommunity.com/id/11122233344455566...,"[{'funny': '', 'posted': 'Posted December 22, ..."
4139,29123,http://steamcommunity.com/id/29123,"[{'funny': '', 'posted': 'Posted March 26.', '..."
...,...,...,...
2721,xXAussieRockXx,http://steamcommunity.com/id/xXAussieRockXx,"[{'funny': '', 'posted': 'Posted July 17, 2015..."
2680,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
17916,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
5855,zeroblade,http://steamcommunity.com/id/zeroblade,"[{'funny': '', 'posted': 'Posted November 30, ..."


In [63]:
duplicated_values(df_reviews,"user_id")["reviews"].duplicated().head(20)

12888    False
5250      True
3133     False
3134      True
4139     False
4138      True
11292    False
4259      True
2638     False
2639      True
2874     False
2872      True
2283     False
2282      True
11730    False
11729     True
8848     False
8850      True
9339     False
9340      True
Name: reviews, dtype: bool

As we can see, the reviews column appears to have duplicated values. So, now before keep going with our process we need to get rid of the duplicates.

In [64]:
df_reviews.drop_duplicates(subset=["user_id"],keep="first",inplace=True)

In [65]:
#Dropping useless columns
df_reviews.drop(columns=["user_url"],inplace=True)

In [66]:
df_reviews.head(5)

,user_id,reviews
0,76561197970982479,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,"[{'funny': '3 people found this review funny',..."


In order to continue with our process we need to unpack what is contained in the reviews column.

In [67]:
df_reviews=expand_json_column(df_reviews,"reviews")
df_reviews.head(5)

,user_id,0,1,2,3,4,5,6,7,8,9
0,76561197970982479,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None
1,js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....","{'funny': '', 'posted': 'Posted September 8, 2...","{'funny': '', 'posted': 'Posted November 29, 2...",None,None,None,None,None,None,None
2,evcentric,"{'funny': '', 'posted': 'Posted February 3.', ...","{'funny': '', 'posted': 'Posted December 4, 20...","{'funny': '', 'posted': 'Posted November 3, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...",None,None,None,None
3,doctr,"{'funny': '', 'posted': 'Posted October 14, 20...","{'funny': '', 'posted': 'Posted July 28, 2012....","{'funny': '', 'posted': 'Posted June 2, 2012.'...","{'funny': '', 'posted': 'Posted June 29, 2014....","{'funny': '', 'posted': 'Posted November 22, 2...","{'funny': '', 'posted': 'Posted February 23, 2...",None,None,None,None
4,maplemage,"{'funny': '3 people found this review funny', ...","{'funny': '1 person found this review funny', ...","{'funny': '2 people found this review funny', ...","{'funny': '', 'posted': 'Posted July 11, 2013....",None,None,None,None,None,None


In [68]:
df_reviews=pd.melt(df_reviews,id_vars=["user_id"],value_vars=list(range(9)),value_name="reviews")

In [69]:
df_reviews.head(5)

,user_id,variable,reviews
0,76561197970982479,0,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,js41637,0,"{'funny': '', 'posted': 'Posted June 24, 2014...."
2,evcentric,0,"{'funny': '', 'posted': 'Posted February 3.', ..."
3,doctr,0,"{'funny': '', 'posted': 'Posted October 14, 20..."
4,maplemage,0,"{'funny': '3 people found this review funny', ..."


In [70]:
general_information(df_reviews)

,column,data_type,missing_values,missing_values_percentage
0,user_id,<class 'str'>,2826,0.01
1,variable,<class 'str'>,0,0.00
2,reviews,<class 'str'>,345220,1.49


We can see that many values in the reviews column are either nan values, or none values or there is no value at all. So, what we can do now, since there is no way for us to fill those values or replace them, is to drop them.

In [71]:
df_reviews.dropna(subset=["reviews"],inplace=True)

In [72]:
general_information(df_reviews)

,column,data_type,missing_values,missing_values_percentage
0,user_id,<class 'str'>,771,0.01
1,variable,<class 'str'>,0,0.00
2,reviews,<class 'str'>,0,0.00


In [73]:
df_reviews.head(5)

,user_id,variable,reviews
0,76561197970982479,0,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,js41637,0,"{'funny': '', 'posted': 'Posted June 24, 2014...."
2,evcentric,0,"{'funny': '', 'posted': 'Posted February 3.', ..."
3,doctr,0,"{'funny': '', 'posted': 'Posted October 14, 20..."
4,maplemage,0,"{'funny': '3 people found this review funny', ..."


In [74]:
#We keep the user_id column for its later use
user_id_series=df_reviews["user_id"].copy()

In [75]:
user_id_series.head(5)

0    76561197970982479
1              js41637
2            evcentric
3                doctr
4            maplemage
Name: user_id, dtype: object

In [76]:
df_reviews=df_reviews["reviews"].apply(pd.Series,dtype="object")

In [77]:
df_reviews.head(5)

,funny,posted,last_edited,item_id,helpful,recommend,review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,,"Posted October 14, 2013.",,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,3 people found this review funny,"Posted April 15, 2014.",,211420,35 of 43 people (81%) found this review helpful,True,Git gud


In [78]:
general_information(df_reviews)

,column,data_type,missing_values,missing_values_percentage
0,funny,<class 'str'>,50176,0.86
1,posted,<class 'str'>,0,0.00
2,last_edited,<class 'str'>,52188,0.90
3,item_id,<class 'str'>,0,0.00
4,helpful,<class 'str'>,0,0.00
5,recommend,<class 'str'>,0,0.00
6,review,<class 'str'>,38,0.00


We see that the "funny" column and the "last_edited" column have a lot of missing values and they won't be useful for later processes, also the helpful column wont be necessary to keep, so we can drop these columns.

In [79]:
df_reviews.drop(columns=["funny","last_edited"],inplace=True)

In [80]:
general_information(df_reviews)

,column,data_type,missing_values,missing_values_percentage
0,posted,<class 'str'>,0,0.0
1,item_id,<class 'str'>,0,0.0
2,helpful,<class 'str'>,0,0.0
3,recommend,<class 'str'>,0,0.0
4,review,<class 'str'>,38,0.0


In [81]:
df_reviews.head(5)

,posted,item_id,helpful,recommend,review
0,"Posted November 5, 2011.",1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,Posted February 3.,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,"Posted October 14, 2013.",250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,"Posted April 15, 2014.",211420,35 of 43 people (81%) found this review helpful,True,Git gud


In [82]:
df_reviews.drop(columns=["helpful"],inplace=True)

In [83]:
df_reviews.head(5)

,posted,item_id,recommend,review
0,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
2,Posted February 3.,248820,True,A suitably punishing roguelike platformer. Wi...
3,"Posted October 14, 2013.",250320,True,This game... is so fun. The fight sequences ha...
4,"Posted April 15, 2014.",211420,True,Git gud


In [84]:
#Changing the posted column to extract the year of the date
df_reviews["posted"] = pd.to_datetime(df_reviews["posted"].astype(str).str.replace(r'Posted |,|\.', '', regex=True), errors='coerce')

In [85]:
df_reviews.head(5)

,posted,item_id,recommend,review
0,2011-11-05,1250,True,Simple yet with great replayability. In my opi...
1,2014-06-24,251610,True,I know what you think when you see this title ...
2,NaT,248820,True,A suitably punishing roguelike platformer. Wi...
3,2013-10-14,250320,True,This game... is so fun. The fight sequences ha...
4,2014-04-15,211420,True,Git gud


In [86]:
df_reviews["year"] = df_reviews["posted"].dt.year.astype("Int64")

In [87]:
df_reviews.head(5)

,posted,item_id,recommend,review,year
0,2011-11-05,1250,True,Simple yet with great replayability. In my opi...,2011
1,2014-06-24,251610,True,I know what you think when you see this title ...,2014
2,NaT,248820,True,A suitably punishing roguelike platformer. Wi...,<NA>
3,2013-10-14,250320,True,This game... is so fun. The fight sequences ha...,2013
4,2014-04-15,211420,True,Git gud,2014


In [88]:
df_reviews.drop(columns=["posted"],inplace=True)

In [89]:
df_reviews.head(5)

,item_id,recommend,review,year
0,1250,True,Simple yet with great replayability. In my opi...,2011
1,251610,True,I know what you think when you see this title ...,2014
2,248820,True,A suitably punishing roguelike platformer. Wi...,<NA>
3,250320,True,This game... is so fun. The fight sequences ha...,2013
4,211420,True,Git gud,2014


In [90]:
df_reviews["user_id"]=user_id_series

In [91]:
df_reviews.head(5)

,item_id,recommend,review,year,user_id
0,1250,True,Simple yet with great replayability. In my opi...,2011,76561197970982479
1,251610,True,I know what you think when you see this title ...,2014,js41637
2,248820,True,A suitably punishing roguelike platformer. Wi...,<NA>,evcentric
3,250320,True,This game... is so fun. The fight sequences ha...,2013,doctr
4,211420,True,Git gud,2014,maplemage


In [92]:
#Saving the final file
reviews_cleaned=df_reviews.to_csv("reviews_cleaned.csv",index=False)